# Data Cleaning on NYC 311 Data

<a id='toc'></a> 

First, I'll bring in the first three million rows of New York City 311 complaint data using pandas.  Then I'll take a look at the data itself, make a few diagnoses, and execute some data cleaning fixes to improve my dataset for analysis!

Here's a brief Table of Contents to make it easier to navigate this document, which did end up getting a little long:

* [Get Insight](#get_insight)
* [What Needs Attention?](#what_needs_attention)
* [Easy Fixes](#easy_fixes)
* [Lat/Long Data](#lat_long)
* [Only Closed Status](#only_closed)
* [Working With Dates](#working_with_dates): *Note: this is long!*
  - [Date Format](#date_format)
  - [Convert and Analyze Date / Time](#convert)
  - [Conversion Problems](#conversion_problems)
  - [Resolving Redundant Date Data](#redundant_date)
  - [Figuring Out Time Between Complaint Creation and Resolution](#time_creation_resolution)
  - [Removing Outlier Rows](#outlier_rows)
  - [Removing Redundant Columns](#redundant_columns)
* [Analyzing Uniqueness](#analyzing_uniqueness)
  
  
We begin by bringing in a few libraries we'll need:

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import sys
from IPython.core.display import display, HTML

In [ ]:
data311 = pd.read_csv("https://data.cityofnewyork.us/resource/fhrw-4uyv.csv?$limit=3000000")

<a id='get_insight'></a> 
## Get Insight 

Let's begin by just looking at the first few rows of our DataFrame.  I've added an option that allows us to see all the columns instead of seeing an ellipsis between the first few and last few columns.  Since some columns have a lot of text, I'm also changing the max colwidth option.  Then we'll use `.info` to give us columnwise details.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
data311.head()

Let's get some overall information about the contents of the data.  Because we're dealing with millions of rows, we have to change a setting to get fully verbose info from `.info`:

In [4]:
pd.options.display.max_info_rows = 5000000
data311.info()

[Back to top](#toc)
<a id='what_needs_attention'></a> 


## What Needs Attention?


A few notes:

* It seems that the data relating to landmark, taxis, vehicles, and bridges are much more sparse than other types of data.  I'm planning on studying road conditions, specifically, potholes, so I'll remove those columns.  Besides, when you see just a few thousand of values out of three million rows, it's clear that this data is extremely sparse and might not be helpful for analysis of the dataset as a whole.  
* Some columns (such as the ones beginning with `location_`) are entirely empty.  So we'll definitely remove those as well.
* I notice that `incident_zip` is a numeric type.  That makes sense, but if I leave zip as numeric, I'll end up potentially treating it as a number instead of a category (for example, by getting the 'median zip code', which does't make sense).  I'll change that to an 'object', or string type.
* There are values that I don't care about, because we're not going to use them -- e.g. the fields that start with `:@computed_region`. These columns aren't documented in the data dictionary and seem to relate to some sort of administrative division of the geography of NYC.  But since I don't know anything about these columns, we might as well delete them.
* `x_coordinate_state_plane` and `y_coordinate_state_plane` are also mysterious.  We have lat/long data, so this additional location data is superfluous.
* Other columns seem to contain redundant data -- for example, `latitude` and `longitude` seem to be contained in `location`.  I like keeping the two values separate for now, so we can probably remove `location`.
* The `status` column seems to be mostly "closed", but I want to ensure that the only data we analyze are closed complaints, so I can work a little to clean up the data using this column.
* The values that I expect to be 'datetime' type are strings.  This includes `created_date`, `closed_date`, etc.  I'll fix those. 
* But I'm also curious about the difference between `closed_date` and `resolution_action_updated_date`, so I'll look at that first!
* I'm curious to see if complaints go up seasonally, so I'll want to make a new variable that bins the complaints by Spring (March, April, May), Summer (June, July, August), Fall (September, October, November), and Winter (December, January, February). 

Let's start there, solve these problems, and look again.  We might find new problems!  Just in case we mess anything up, we'll do all our cleaning in a new DataFrame called `clean_311`.

[Back to top](#toc)
<a id='easy_fixes'></a> 


## Easy Fixes

We'll begin by removing some columns, columns, keeping only those missing fewer than 10% of their values.  We set the threshold at a percentage of the number of rows in the data frame (the first shape metric).  We'll also change the `incident_zip` to a string.  We can also drop some columns we know we're not going to use.  Those operations are simple enough that we'll do them all before checking in again on the DataFrame.

In [5]:
clean_311 = data311.dropna(thresh=(0.10 * data311.shape[0]), axis=1).copy()

In [6]:
clean_311.loc[:,'incident_zip'] = clean_311['incident_zip'].astype(str)

In [7]:
clean_311.drop(columns=list(clean_311.filter(regex=':@computed')), inplace=True)
clean_311.drop(columns=["x_coordinate_state_plane", "y_coordinate_state_plane"], inplace=True)

Let's take a peek at what `clean_311` looks like now, as far as data types and number of columns (and values in those columns):

In [8]:
clean_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 31 columns):
unique_key                        3000000 non-null int64
created_date                      3000000 non-null object
closed_date                       2831174 non-null object
agency                            3000000 non-null object
agency_name                       3000000 non-null object
complaint_type                    3000000 non-null object
descriptor                        2992582 non-null object
location_type                     2323029 non-null object
incident_zip                      3000000 non-null object
incident_address                  2526406 non-null object
street_name                       2526397 non-null object
cross_street_1                    2631223 non-null object
cross_street_2                    2630298 non-null object
intersection_street_1             465199 non-null object
intersection_street_2             465225 non-null object
address_type        

[Back to top](#toc)
<a id=lat_long'></a> 

## Lat/Long Data

I think it's true that `location` is simply a concatenation of `latitude` and `longitude`.  See what I mean?


In [9]:
clean_311[["latitude", "longitude", "location"]].head(30)

,latitude,longitude,location
0,40.763866,-73.932628,POINT (-73.932628238422 40.763866316425)
1,40.654613,-73.960309,POINT (-73.960309163909 40.654612911819)
2,40.856550,-73.927661,POINT (-73.927661412425 40.856549859021)
3,40.701877,-73.825883,POINT (-73.825882694038 40.701876580788)
4,40.671582,-73.932696,POINT (-73.932696223379 40.671582019618)
5,40.681673,-73.877768,POINT (-73.877767625521 40.681673234879)
6,40.628993,-73.967948,POINT (-73.967947534428 40.628992720751)
7,40.685022,-73.981972,POINT (-73.981971790081 40.685022237661)
8,40.869644,-73.863126,POINT (-73.863125800093 40.869643952747)
9,40.671182,-73.933688,POINT (-73.93368797325 40.671181858104)


Still, before removing `location`, it pays to check to see if my intuition is correct.  We might also consider getting the more precise latitude and longitude from the `location` column -- there is more decimal precision in that data.  Let's first see if it's true that all the `location` data follows the same pattern I see right now: 

In [10]:
clean_311['location'].str.match('POINT \(-7\d\.\d+ \d{2}\.\d+\)', na=False).value_counts()

True     2999980
False         20
Name: location, dtype: int64

WOW, only 20 rows where that's not the case!  Let's check out which ones those are!

In [11]:
clean_311[~clean_311['location'].str.match('POINT \(-7\d\.\d+ \d{2}\.\d+\)', na=False)]

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,facility_type,status,due_date,resolution_description,resolution_action_updated_date,community_board,bbl,borough,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location
125251,18417851,2010-08-05T21:25:48.000,2010-10-05T06:15:23.000,DOHMH,Department of Health and Mental Hygiene,Food Establishment,Rodents/Insects/Garbage,Restaurant/Bar/Deli/Bakery,10011.0,190 8 AVENUE,8 AVENUE,WEST 19 STREET,WEST 20 STREET,NaN,NaN,ADDRESS,NEW YORK,NaN,Closed,2010-10-04T21:25:48.000,The Department of Health and Mental Hygiene has sent official written notification to the Owner/Landlord warning them of potential violations and instructing them to correct the situation. If the...,2010-10-05T00:00:00.000,04 MANHATTAN,1.007690e+09,MANHATTAN,PHONE,Unspecified,MANHATTAN,40.743196,-74.0,POINT (-74 40.743196117914)
203014,18542923,2010-08-23T11:42:36.000,2010-09-17T00:00:00.000,DOB,Department of Buildings,Plumbing,Plumbing-Defective/Leaking/Not Maintained,NaN,10011.0,190 8 AVENUE,8 AVENUE,WEST 19 STREET,WEST 20 STREET,NaN,NaN,ADDRESS,NEW YORK,NaN,Closed,NaN,The Department of Buildings investigated this complaint and determined that no further action was necessary.,2010-09-17T00:00:00.000,04 MANHATTAN,1.007690e+09,MANHATTAN,UNKNOWN,Unspecified,MANHATTAN,40.743196,-74.0,POINT (-74 40.743196117914)
680591,19228952,2010-11-30T17:58:48.000,2011-01-30T06:15:48.000,DOHMH,Department of Health and Mental Hygiene,Food Establishment,Rodents/Insects/Garbage,Restaurant/Bar/Deli/Bakery,10011.0,190 8 AVENUE,8 AVENUE,WEST 19 STREET,WEST 20 STREET,NaN,NaN,ADDRESS,NEW YORK,NaN,Closed,2011-01-29T17:58:48.000,The Department of Health and Mental Hygiene has sent official written notification to the Owner/Landlord warning them of potential violations and instructing them to correct the situation. If the...,2011-01-30T06:15:48.000,04 MANHATTAN,1.007690e+09,MANHATTAN,PHONE,Unspecified,MANHATTAN,40.743196,-74.0,POINT (-74 40.743196117914)
866290,19534957,2011-01-04T06:28:23.000,2011-01-04T21:23:00.000,DOT,Department of Transportation,Street Condition,Pothole,NaN,11214.0,7519 NEW UTRECHT AVENUE,NEW UTRECHT AVENUE,BAY RIDGE PARKWAY,76 STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,Closed,NaN,The Department of Transportation inspected this complaint and repaired the problem.,2011-01-04T21:23:00.000,Unspecified BROOKLYN,3.062250e+09,BROOKLYN,UNKNOWN,Unspecified,BROOKLYN,40.615548,-74.0,POINT (-74 40.61554771179)
1005618,19734218,2011-02-01T01:20:00.000,2011-02-02T09:36:00.000,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,10011.0,190 8 AVENUE,8 AVENUE,WEST 19 STREET,WEST 20 STREET,NaN,NaN,ADDRESS,NEW YORK,NaN,Closed,NaN,Service Request status for this request is available on the Department of Transportationâs website. Please click the âLearn Moreâ link below.,2011-02-02T09:36:00.000,04 MANHATTAN,1.007690e+09,MANHATTAN,UNKNOWN,Unspecified,MANHATTAN,40.743196,-74.0,POINT (-74 40.743196117914)
1073786,19830124,2011-02-10T15:02:58.000,2011-02-14T00:00:00.000,DOB,Department of Buildings,General Construction/Plumbing,Failure To Maintain,NaN,10011.0,190 8 AVENUE,8 AVENUE,WEST 19 STREET,WEST 20 STREET,NaN,NaN,ADDRESS,NEW YORK,NaN,Closed,NaN,The Department of Buildings investigated this complaint and issued an Office of Administrative Trials and Hearings (OATH) summons.,2011-02-14T00:00:00.000,04 MANHATTAN,1.007690e+09,MANHATTAN,UNKNOWN,Unspecified,MANHATTAN,40.743196,-74.0,POINT (-74 40.743196117914)
1273386,20113295,2011-03-26T22:39:12.000,2011-03-27T00:12:39.000,NYPD,New York City Police Department,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,11219.0,5104 10 AVENUE,10 AVENUE,51 STREET,52 STREET,NaN,NaN,ADDRESS,BROOKLYN,Precinct,Closed,2011-03-27T06:39:12.000,The Police Department responded to the

OK, that's fine -- these didn't match because they were missing a decimal, and I put a decimal point in the regex.  We've established that `location` has the same data (if a bit more precise!) as `latitude` and `longitude`.  Let's extract this more precise lat/long data and use those as the lat/long columns, instead of what we were provided originally.

In [12]:
new_lat_long = (clean_311['location'].str.extract('.+(\-\d{2}\.*\d*) (\d{2}\.*\d*).+')).astype(float)
new_lat_long.head()

,0,1
0,-73.932628,40.763866
1,-73.960309,40.654613
2,-73.927661,40.856550
3,-73.825883,40.701877
4,-73.932696,40.671582


Fantastic, now we can replace the lat/long in `clean_311` with this more precise data!

In [13]:
clean_311.loc[:, 'longitude'] = new_lat_long[0]
clean_311.loc[:, 'latitude'] = new_lat_long[1]

Let's drop `location`:

In [14]:
clean_311.drop(columns="location", inplace = True)

And let's peek at our data so far:

In [15]:
clean_311.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,facility_type,status,due_date,resolution_description,resolution_action_updated_date,community_board,bbl,borough,open_data_channel_type,park_facility_name,park_borough,latitude,longitude
0,17607569,2010-07-08T00:00:00.000,2011-09-01T00:00:00.000,HPD,Department of Housing Preservation and Development,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,11106.0,21-20 33 ROAD,33 ROAD,21 STREET,23 STREET,NaN,NaN,ADDRESS,ASTORIA,NaN,Closed,NaN,"The Department of Housing Preservation and Development was not able to gain access to inspect the following conditions. The complaint has been closed. If the condition still exists, please file a ...",2011-09-01T00:00:00.000,0 Unspecified,4.005570e+09,Unspecified,UNKNOWN,Unspecified,Unspecified,40.763866,-73.932628
1,17607570,2010-07-08T00:00:00.000,2010-07-29T00:00:00.000,HPD,Department of Housing Preservation and Development,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,11226.0,170 WOODRUFF AVENUE,WOODRUFF AVENUE,EAST 21 STREET,CLARKSON AVENUE,NaN,NaN,ADDRESS,BROOKLYN,NaN,Closed,NaN,"The Department of Housing Preservation and Development was not able to gain access to inspect the following conditions. The complaint has been closed. If the condition still exists, please file a ...",2010-07-29T00:00:00.000,0 Unspecified,3.050630e+09,Unspecified,UNKNOWN,Unspecified,Unspecified,40.654613,-73.960309
2,17607571,2010-07-08T00:00:00.000,2010-07-22T00:00:00.000,HPD,Department of Housing Preservation and Development,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,10040.0,558 WEST 193 STREET,WEST 193 STREET,AUDUBON AVENUE,FT GEORGE AVENUE,NaN,NaN,ADDRESS,NEW YORK,NaN,Closed,NaN,The Department of Housing Preservation and Development inspected the following conditions. Violations were issued. Information about specific violations is available at www.nyc.gov/hpd.,2010-07-22T00:00:00.000,0 Unspecified,1.021610e+09,Unspecified,UNKNOWN,Unspecified,Unspecified,40.856550,-73.927661
3,17607572,2010-07-08T00:00:00.000,2010-08-12T00:00:00.000,HPD,Department of Housing Preservation and Development,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,11418.0,86-48 125 STREET,125 STREET,HILLSIDE AVENUE,JAMAICA AVENUE,NaN,NaN,ADDRESS,RICHMOND HILL,NaN,Closed,NaN,"The Department of Housing Preservation and Development was not able to gain access to inspect the following conditions. The complaint has been closed. If the condition still exists, please file a ...",2010-08-12T00:00:00.000,0 Unspecified,4.092780e+09,Unspecified,UNKNOWN,Unspecified,Unspecified,40.701877,-73.825883
4,17607573,2010-07-08T00:00:00.000,2010-07-14T00:00:00.000,HPD,Department of Housing Preservation and Development,ELECTRIC,ELECTRIC-SUPPLY,RESIDENTIAL BUILDING,11213.0,1381 STERLING PLACE,STERLING PLACE,SCHENECTADY AVENUE,UTICA AVENUE,NaN,NaN,ADDRESS,BROOKLYN,NaN,Closed,NaN,The Department of Housing Preservation and Development inspected the following conditions. No violations were issued. The complaint has been closed.,2010-07-14T00:00:00.000,0 Unspecified,3.013720e+09,Unspecified,UNKNOWN,Unspecified,Unspecified,40.671582,-73.932696


[Back to top](#toc)
<a id=only_closed'></a> 

## Only "Closed" Status

For our analysis, we're only interested in complaints that are finished up -- we want to look at final data, not complaints that are still in the middle of being worked.  Let's take a look at the status values:

In [16]:
clean_311['status'].value_counts()

Closed              2774723
Open                 144180
Pending               52486
Assigned              28294
Started                 301
Unspecified               9
Closed - Testing          4
Unassigned                3
Name: status, dtype: int64

Let's remove all the rows that aren't "closed", and then remove the status column altogether!

In [17]:
clean_311.drop(clean_311[clean_311['status'] != "Closed"].index, axis=0, inplace=True)
clean_311.drop(columns='status', inplace=True)

In [18]:
clean_311.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2774723 entries, 0 to 2999999
Data columns (total 29 columns):
unique_key                        2774723 non-null int64
created_date                      2774723 non-null object
closed_date                       2774005 non-null object
agency                            2774723 non-null object
agency_name                       2774723 non-null object
complaint_type                    2774723 non-null object
descriptor                        2767358 non-null object
location_type                     2151084 non-null object
incident_zip                      2774723 non-null object
incident_address                  2340895 non-null object
street_name                       2340888 non-null object
cross_street_1                    2442534 non-null object
cross_street_2                    2441723 non-null object
intersection_street_1             425513 non-null object
intersection_street_2             425539 non-null object
address_type        

[Back to top](#toc)
<a id='working_with_dates'></a> 

## Working With Dates

<a id='date_format'></a>
### Date Format

Let's see about our dates.  Let's look at them to make sure they are all in the same format:

In [19]:
clean_311[['created_date', 'closed_date', 'due_date', 'resolution_action_updated_date']].head()

,created_date,closed_date,due_date,resolution_action_updated_date
0,2010-07-08T00:00:00.000,2011-09-01T00:00:00.000,NaN,2011-09-01T00:00:00.000
1,2010-07-08T00:00:00.000,2010-07-29T00:00:00.000,NaN,2010-07-29T00:00:00.000
2,2010-07-08T00:00:00.000,2010-07-22T00:00:00.000,NaN,2010-07-22T00:00:00.000
3,2010-07-08T00:00:00.000,2010-08-12T00:00:00.000,NaN,2010-08-12T00:00:00.000
4,2010-07-08T00:00:00.000,2010-07-14T00:00:00.000,NaN,2010-07-14T00:00:00.000


Hmmm... I'd really like to see more `due_date` data, so let me order by that column:

In [20]:
clean_311[['created_date', 'closed_date', 'due_date', 'resolution_action_updated_date']].sort_values('due_date').head()

,created_date,closed_date,due_date,resolution_action_updated_date
5297,2010-07-09T02:34:24.000,2010-07-09T04:58:58.000,2010-07-09T10:34:24.000,2010-07-09T00:00:00.000
5614,2010-07-09T02:39:34.000,2010-07-09T21:00:49.000,2010-07-09T10:39:34.000,2010-07-09T00:00:00.000
5298,2010-07-09T02:42:53.000,2010-07-09T05:00:01.000,2010-07-09T10:42:53.000,2010-07-09T00:00:00.000
5532,2010-07-09T02:43:11.000,2010-07-09T05:19:08.000,2010-07-09T10:43:11.000,2010-07-09T00:00:00.000
5201,2010-07-09T02:51:23.000,2010-07-09T04:31:34.000,2010-07-09T10:51:23.000,2010-07-09T00:00:00.000


What's interesting is that there's a lot of similarity across the board with these dates.  In the first view of the DataFrame, I see that `closed_date` and `resolution_action_updated_date` seem to be on the same day.  In this new view, sorted by `due_date`, it looks like sometimes all the dates are on the same day!  We also know (from above) that only around a third of the complaints get due dates. 

Let's get a little more insight into these dates.  I'll write a short function to do a couple of things.

[Back to top](#toc)
<a id='convert'></a>
### Convert and Analyze Date / Time

In [21]:
def fix_and_show_dates (col_name):
    display(HTML("<div class='alert alert-block alert-info'>Attempting to Convert <code>" + col_name + "</code> to Datetime.</div>"))
    try:
        clean_311[col_name] = pd.to_datetime(clean_311[col_name])
    except: 
        display(HTML("<div class='alert alert-block alert-danger'>Unexpected error:" + str(sys.exc_info()) + "</div>"))
        return
    display(HTML("<div class='alert alert-block alert-success'>Strings Succesfully Converted to Datetime.</div> <br/> <strong>Statistical Description of <code>" + col_name + "</code>:</strong>"))
    print(clean_311[col_name].describe())

We'll start with `created_date`.

In [22]:
fix_and_show_dates('created_date')

count                 2774723
unique                1419370
top       2012-01-04 00:00:00
freq                     5922
first     2010-01-04 00:00:00
last      2012-04-16 21:44:00
Name: created_date, dtype: object


OK, so that looks normal.  All the dates are from around 2010 and later, which makes sense.  What about `closed_date`?

In [23]:
fix_and_show_dates('closed_date')

count                 2774005
unique                 969586
top       2010-12-09 00:00:00
freq                     6539
first     1900-01-01 00:00:00
last      2201-03-25 00:00:00
Name: closed_date, dtype: object


Hmmm... some weird dates there, like 1900 and 2201!  We'll address these at the end.

In [24]:
fix_and_show_dates('due_date')

count                  898209
unique                 860597
top       2010-12-28 10:42:41
freq                       44
first     2010-07-09 10:34:24
last      2016-06-13 16:59:37
Name: due_date, dtype: object


This looks pretty good.  Still, there are less than 1M of them in a 3M row dataset, so we should consider if we even want this field.  We don't actually care about when work was 'due', we care about when it was completed!

Finally, let's look at `resolution_action_updated_date`.

In [25]:
fix_and_show_dates('resolution_action_updated_date')

count                 2765861
unique                 813859
top       2010-11-15 00:00:00
freq                     6981
first     2010-01-04 00:00:00
last      2120-09-10 21:20:00
Name: resolution_action_updated_date, dtype: object


Again, a weird date!  We'll solve this one along with the ones in `closed_date`

[Back to top](#toc)
<a id='conversion_problems'></a>
### Conversion Problems

Although it didn't happen this time, in the past, we've experienced an "OutOfBoundsDatetime" error that prevented conversion.  In a case like that, we can still sort a string / "object" type of date in alphabetical order to see the lowest and highest dates.  That comes in handy if we have wildly improbable dates like the year 3000 that are "out of bounds".

We might also see some successful conversions that still don't make sense -- like an early 20th century timestamp, which seems improbable at best.  Therefore, across the board on dates, let's get rid of dates that are before 2007 or after today.  

In [26]:
for col in ['created_date', 'closed_date', 'due_date', 'resolution_action_updated_date']:
    clean_311.loc[clean_311[col] < '2007-01-01', col] = pd.NaT
    clean_311.loc[clean_311[col] > pd.Timestamp(dt.date.today())] = pd.NaT

[Back to top](#toc)
<a id='redundant_date'></a>
### Resolving Redundant Date Data

We still haven't figured out which dates are worth keeping and what the relationship between the dates are... let's do that next.  First of all, I think the `closed_date` and `resolution_action_updated_date` occur on the same day.  Am I right?

In [27]:
(clean_311['closed_date'].dt.day == clean_311['resolution_action_updated_date'].dt.day).value_counts()

True     2682729
False      91994
dtype: int64

Wow, it certainly looks true almost all the time.  Let's take a peek at where it's *not* true.  We'll sort and look at the top and bottom of a couple of views to make sure we get a diverse look.

In [28]:
clean_311.loc[clean_311['closed_date'].dt.day != clean_311['resolution_action_updated_date'].dt.day, 
              ['closed_date','resolution_action_updated_date']].sort_values('closed_date').head()

,closed_date,resolution_action_updated_date
149140,2010-06-17 09:30:00,2010-08-09 08:45:00
4097,2010-07-08 09:31:00,2010-07-07 00:00:00
3228,2010-07-08 09:47:00,2010-07-07 00:00:00
3229,2010-07-08 09:48:00,2010-07-07 00:00:00
3230,2010-07-08 09:49:00,2010-07-07 00:00:00


In [29]:
clean_311.loc[clean_311['closed_date'].dt.day != clean_311['resolution_action_updated_date'].dt.day, 
              ['closed_date','resolution_action_updated_date']].sort_values('closed_date').tail()

,closed_date,resolution_action_updated_date
2964219,NaT,2013-12-06 09:10:55
2979795,NaT,2013-07-24 00:00:00
2988879,NaT,2013-08-13 13:46:47
2992709,NaT,2012-04-24 12:46:59
2995306,NaT,2013-12-05 00:00:00


In [30]:
clean_311.loc[clean_311['closed_date'].dt.day != clean_311['resolution_action_updated_date'].dt.day, 
              ['closed_date','resolution_action_updated_date']].sort_values('resolution_action_updated_date').tail()

,closed_date,resolution_action_updated_date
2999741,2012-04-14 20:15:39,NaT
2999789,2012-04-15 02:05:39,NaT
2999802,2012-04-15 01:17:49,NaT
2999827,2012-04-14 21:57:33,NaT
2999834,2012-04-14 08:35:49,NaT


What's really interesting is that it seems as though sometimes we have the `closed_date`, sometimes we have the `resolution_action_updated_date`, sometimes neither -- although that's not obvious here, it's a good probability -- , and sometimes both.  I think the two dates are *generally* capturing the same thing (because we've seen that they line up, mostly on the same day or very nearly so).  So I'll create a new column to capture whichever one exists, and if they both exist, choose the `closed_date`.

In [31]:
clean_311['resolved_date'] = np.where(clean_311['closed_date'].notnull(), clean_311['closed_date'], clean_311['resolution_action_updated_date'])

Let's peek to see if this looks right.  We should have the a higher 'count' for `resolved_date` than for `closed_date`, as it will handle missing values in the `closed_date` column.  

In [32]:
clean_311[['closed_date', 'resolution_action_updated_date', 'resolved_date']].describe()

,closed_date,resolution_action_updated_date,resolved_date
count,2773989,2765846,2774672
unique,969582,813845,969719
top,2010-12-09 00:00:00,2010-11-15 00:00:00,2010-12-09 00:00:00
freq,6539,6981,6539
first,2010-01-04 10:32:54,2010-01-04 00:00:00,2010-01-04 10:32:54
last,2019-06-25 00:00:00,2019-06-25 00:00:00,2019-06-25 00:00:00


Let's peek at where there are nulls in `closed_date`.  Did our code handle this?

In [33]:
clean_311.loc[clean_311['closed_date'].isnull(),
              ['closed_date','resolution_action_updated_date', 'resolved_date']].head(50)

,closed_date,resolution_action_updated_date,resolved_date
2296,NaT,2012-04-02 15:36:38,2012-04-02 15:36:38
25906,NaT,2012-04-02 15:36:38,2012-04-02 15:36:38
29787,NaT,2012-04-02 15:36:38,2012-04-02 15:36:38
39389,NaT,NaT,NaT
66732,NaT,2012-04-02 15:36:38,2012-04-02 15:36:38
88243,NaT,2012-04-02 15:36:38,2012-04-02 15:36:38
98675,NaT,2012-04-02 15:36:38,2012-04-02 15:36:38
102103,NaT,NaT,NaT
133037,NaT,2012-04-02 15:36:38,2012-04-02 15:36:38
133846,NaT,NaT,NaT


It did!  So now, we're nearly ready to remove `closed_date`, `due_date`, and `resolution_action_updated_date`.  First, we will create a new column, `days_to_close`, which is just simple math that calculates the number of days between `created_date` and `resolved_date`, each of those with the time stripped off so that the fact of time being recorded or not doesn't create some weird artifacts (e.g. Friday with no time given minus Friday at 6am).


[Back to top](#toc)
<a id="time_creation_resolution"></a>
### Figuring Out Time From Complaint Creation to Resolution

In [34]:
clean_311['days_to_close'] = (clean_311['resolved_date'].dt.date - clean_311['created_date'].dt.date).dt.days

Let's peek at the statistics of all our numeric variables (and at this point, there should only be one!)

In [35]:
clean_311['days_to_close'].describe()

count    2.774672e+06
mean     1.916779e+01
std      9.445253e+01
min      0.000000e+00
25%      1.000000e+00
50%      4.000000e+00
75%      1.200000e+01
max      3.260000e+03
Name: days_to_close, dtype: float64

Great, there are no "negative days".  The close is always after the open, which is a good sign!  

[Back to top](#toc)
<a id='redundant_columns'></a>
### Removing Redundant Columns

Wow, that was a lot of date work, but the great thing is it allows us to remove the columns we don't care about any more.  We really only want to save the `created_date` and `days_to_close` variables -- the other date columns can go away.  Let's do that, then assess where we are!

In [36]:
clean_311.drop(columns=['closed_date', 'due_date', 'resolution_action_updated_date'], inplace=True)

In [37]:
clean_311.head()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,facility_type,resolution_description,community_board,bbl,borough,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,resolved_date,days_to_close
0,17607569,2010-07-08,HPD,Department of Housing Preservation and Development,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,11106.0,21-20 33 ROAD,33 ROAD,21 STREET,23 STREET,NaN,NaN,ADDRESS,ASTORIA,NaN,"The Department of Housing Preservation and Development was not able to gain access to inspect the following conditions. The complaint has been closed. If the condition still exists, please file a ...",0 Unspecified,4.00557e+09,Unspecified,UNKNOWN,Unspecified,Unspecified,40.7639,-73.9326,2011-09-01,420.0
1,17607570,2010-07-08,HPD,Department of Housing Preservation and Development,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,11226.0,170 WOODRUFF AVENUE,WOODRUFF AVENUE,EAST 21 STREET,CLARKSON AVENUE,NaN,NaN,ADDRESS,BROOKLYN,NaN,"The Department of Housing Preservation and Development was not able to gain access to inspect the following conditions. The complaint has been closed. If the condition still exists, please file a ...",0 Unspecified,3.05063e+09,Unspecified,UNKNOWN,Unspecified,Unspecified,40.6546,-73.9603,2010-07-29,21.0
2,17607571,2010-07-08,HPD,Department of Housing Preservation and Development,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,10040.0,558 WEST 193 STREET,WEST 193 STREET,AUDUBON AVENUE,FT GEORGE AVENUE,NaN,NaN,ADDRESS,NEW YORK,NaN,The Department of Housing Preservation and Development inspected the following conditions. Violations were issued. Information about specific violations is available at www.nyc.gov/hpd.,0 Unspecified,1.02161e+09,Unspecified,UNKNOWN,Unspecified,Unspecified,40.8565,-73.9277,2010-07-22,14.0
3,17607572,2010-07-08,HPD,Department of Housing Preservation and Development,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,11418.0,86-48 125 STREET,125 STREET,HILLSIDE AVENUE,JAMAICA AVENUE,NaN,NaN,ADDRESS,RICHMOND HILL,NaN,"The Department of Housing Preservation and Development was not able to gain access to inspect the following conditions. The complaint has been closed. If the condition still exists, please file a ...",0 Unspecified,4.09278e+09,Unspecified,UNKNOWN,Unspecified,Unspecified,40.7019,-73.8259,2010-08-12,35.0
4,17607573,2010-07-08,HPD,Department of Housing Preservation and Development,ELECTRIC,ELECTRIC-SUPPLY,RESIDENTIAL BUILDING,11213.0,1381 STERLING PLACE,STERLING PLACE,SCHENECTADY AVENUE,UTICA AVENUE,NaN,NaN,ADDRESS,BROOKLYN,NaN,The Department of Housing Preservation and Development inspected the following conditions. No violations were issued. The complaint has been closed.,0 Unspecified,3.01372e+09,Unspecified,UNKNOWN,Unspecified,Unspecified,40.6716,-73.9327,2010-07-14,6.0


[Back to top](#toc)
<a id='seasons'></a>
### Making a Seasons Variable

I'm interested in adding a variable that gives the season in which something occurred.

In [39]:
def season(date):
    if date.month in ([3, 4, 5]):
        val = 'Spring'
    elif date.month in ([6, 7, 8]):
        val = 'Summer'
    elif date.month in ([9, 10, 11]):
        val = 'Autumn'
    elif date.month in ([12, 1, 2]):
        val = 'Winter'
    else:
        val = "Unspecified"
    return val

clean_311['season'] = clean_311['created_date'].apply(season)

In [40]:
clean_311['season'].value_counts()

Autumn         844428
Winter         835498
Summer         599794
Spring         494988
Unspecified        15
Name: season, dtype: int64

[Back to top](#toc)
<a id='analyzing_uniqueness'></a>
## Analyzing Uniqueness

Do any of the columns that are left have only a few values in them, which might indicate (1) a reason to delete the column, or (2) a place to do aggregation / grouping?  Let's check:

In [41]:
clean_311.nunique()

unique_key                2774708
created_date              1419362
agency                         15
agency_name                  1066
complaint_type                155
descriptor                    875
location_type                  98
incident_zip                  410
incident_address           490220
street_name                 15008
cross_street_1              16630
cross_street_2              16777
intersection_street_1        8967
intersection_street_2        9085
address_type                    4
city                          114
facility_type                   4
resolution_description        954
community_board                77
bbl                        366797
borough                         6
open_data_channel_type          4
park_facility_name           1649
park_borough                    6
latitude                   511932
longitude                  511934
resolved_date              969719
days_to_close                2434
season                          5
dtype: int64

I'm surprised to see so many unique values in `city`.  What's going on here?

In [42]:
clean_311['city'].value_counts()

BROOKLYN                  900410
NEW YORK                  554198
BRONX                     553256
STATEN ISLAND             136138
JAMAICA                    66217
FLUSHING                   51674
ASTORIA                    39195
RIDGEWOOD                  29956
WOODSIDE                   20728
FAR ROCKAWAY               20261
CORONA                     19819
ELMHURST                   18729
FOREST HILLS               17022
OZONE PARK                 16680
QUEENS VILLAGE             16593
EAST ELMHURST              15940
JACKSON HEIGHTS            15741
SOUTH RICHMOND HILL        14517
SOUTH OZONE PARK           13733
LONG ISLAND CITY           13422
FRESH MEADOWS              13060
RICHMOND HILL              11865
MASPETH                    11608
WOODHAVEN                  11338
SPRINGFIELD GARDENS        11253
BAYSIDE                    11073
SAINT ALBANS               10796
REGO PARK                  10321
HOLLIS                     10095
MIDDLE VILLAGE              9941
          

OK, so it seems that sometimes this is a borough, sometimes a neighborhood indicator (including Long Island), and sometimes an indicator of a city outside of New York proper.  For example, I'd love to understand 'BUFFALO'!

In [43]:
clean_311[clean_311['city'] == "BUFFALO"]

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,facility_type,resolution_description,community_board,bbl,borough,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,resolved_date,days_to_close,season
2757327,22708134,2012-02-13 15:30:04,DCA,Department of Consumer Affairs,Consumer Complaint,Billing Dispute,NaN,14210.0,726 EXCHANGE ST,EXCHANGE ST,NaN,NaN,NaN,NaN,ADDRESS,BUFFALO,NaN,"The Department of Consumer Affairs mailed you a complaint form. If you received the form and have not yet completed it, please do so. Be sure to follow the instructions on the form and return it t...",0 Unspecified,3.05417e+09,Unspecified,PHONE,Unspecified,Unspecified,40.6336,-73.9711,2012-02-29 01:06:47,16.0,Winter


Hmmm... is this some sort of complaint on the part of a Buffalo resident about a New York company or government agency?  "Non-Delivery", e.g. maybe a NYC business cheated someone from Buffalo?  Let's look more closely:

In [44]:
clean_311[clean_311['city'] == "BUFFALO"].transpose()

,2757327
unique_key,22708134
created_date,2012-02-13 15:30:04
agency,DCA
agency_name,Department of Consumer Affairs
complaint_type,Consumer Complaint
descriptor,Billing Dispute
location_type,NaN
incident_zip,14210.0
incident_address,726 EXCHANGE ST
street_name,EXCHANGE ST


Well, that's interesting, but none of what we've seen makes me want to remove `city` or do much work on it for now.  We can leave `city` alone until we get deeper into analysis!  There are other variables I want to check out, because they have only a few categories:  

* address_type
* facility_type
* open_data_channel_type
* borough (6, instead of 5?)
* park_borough (6, instead of 5?)


In [45]:
clean_311['address_type'].value_counts()

ADDRESS         2265054
INTERSECTION     425543
BLOCKFACE         67679
PLACENAME          2308
Name: address_type, dtype: int64

OK, nothing in `address_type` really asks for reduction or cleanup.

In [46]:
clean_311['facility_type'].value_counts()

Precinct           491348
DSNY Garage         80343
School               3729
School District      1096
Name: facility_type, dtype: int64

Interesting!  If we were doing analysis on DSNY (Department of Sanitation), it might be interesting to look at just the DSNY rows, or similarly for police precincts if we were zooming in on NYPD.  Out of curiosity, let's take a look at each of these types more closely.

In [47]:
clean_311[clean_311['facility_type'] == "Precinct"].head()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,facility_type,resolution_description,community_board,bbl,borough,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,resolved_date,days_to_close,season
4164,17613657,2010-07-09 20:26:28,NYPD,New York City Police Department,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,11375.0,NaN,NaN,NaN,NaN,64 ROAD,GRAND CENTRAL PARKWAY,INTERSECTION,FOREST HILLS,Precinct,NaN,06 QUEENS,NaN,QUEENS,PHONE,Unspecified,QUEENS,40.7336,-73.8448,2010-07-09 21:12:58,0.0,Summer
4204,17613194,2010-07-09 03:21:36,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11209.0,317 100 STREET,100 STREET,3 AVENUE,4 AVENUE,NaN,NaN,ADDRESS,BROOKLYN,Precinct,NaN,10 BROOKLYN,3.06134e+09,BROOKLYN,PHONE,Unspecified,BROOKLYN,40.6135,-74.0349,2010-07-09 04:49:49,0.0,Summer
4205,17613196,2010-07-09 06:19:13,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,10009.0,188 AVENUE C,AVENUE C,EAST 11 STREET,EAST 12 STREET,NaN,NaN,ADDRESS,NEW YORK,Precinct,NaN,03 MANHATTAN,1.00382e+09,MANHATTAN,PHONE,Unspecified,MANHATTAN,40.7268,-73.9768,2010-07-09 07:57:54,0.0,Summer
4206,17613197,2010-07-09 06:47:10,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,11204.0,1929 60 STREET,60 STREET,19 AVENUE,20 AVENUE,NaN,NaN,ADDRESS,BROOKLYN,Precinct,NaN,12 BROOKLYN,3.05513e+09,BROOKLYN,PHONE,Unspecified,BROOKLYN,40.6203,-73.9843,2010-07-09 08:29:08,0.0,Summer
4207,17613198,2010-07-09 06:57:43,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,11694.0,209 BEACH 125 STREET,BEACH 125 STREET,ROCKAWAY BEACH BOULEVARD,NEWPORT AVENUE,NaN,NaN,ADDRESS,ROCKAWAY PARK,Precinct,NaN,14 QUEENS,4.16221e+09,QUEENS,PHONE,Unspecified,QUEENS,40.5777,-73.8449,2010-07-09 07:53:49,0.0,Summer


In [48]:
clean_311[clean_311['facility_type'] == "DSNY Garage"].head()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,facility_type,resolution_description,community_board,bbl,borough,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,resolved_date,days_to_close,season
974,17609231,2010-07-08 08:00:00,DSNY,BCC - Queens East,Sanitation Condition,12 Dead Animals,Sidewalk,11429.0,98-13 211 STREET,211 STREET,LONG ISLAND RAILROAD,99 AVENUE,NaN,NaN,ADDRESS,QUEENS VILLAGE,DSNY Garage,The Department of Sanitation removed the items.,13 QUEENS,4.10836e+09,QUEENS,PHONE,Unspecified,QUEENS,40.714,-73.7488,2010-07-09 12:00:00,1.0,Summer
1424,17608973,2010-07-08 06:21:00,DSNY,BCC - Staten Island,Sanitation Condition,12 Dead Animals,Sidewalk,10309.0,3 BOMBAY STREET,BOMBAY STREET,CORRELL AVENUE,SHIEL AVENUE,NaN,NaN,ADDRESS,STATEN ISLAND,DSNY Garage,"The Department of Sanitation has investigated the complaint and addressed the issue. If the problem persists, call 311 to enter a new complaint. If you are outside of New York City, please call (2...",03 STATEN ISLAND,5.07035e+09,STATEN ISLAND,PHONE,Unspecified,STATEN ISLAND,40.5459,-74.2136,2010-07-10 12:00:00,2.0,Summer
1425,17608974,2010-07-08 08:02:00,DSNY,BCC - Brooklyn South,Missed Collection (All Materials),2 Bulk-Missed Collection,Sidewalk,11234.0,4609 AVENUE M,AVENUE M,EAST 46 STREET,SCHENECTADY AVENUE,NaN,NaN,ADDRESS,BROOKLYN,DSNY Garage,The Department of Sanitation picked up the items and determined that the missed collection complaint was not warranted.,18 BROOKLYN,3.07844e+09,BROOKLYN,PHONE,Unspecified,BROOKLYN,40.6211,-73.9308,2010-07-09 12:00:00,1.0,Summer
1428,17608977,2010-07-08 10:52:00,DSNY,BCC - Brooklyn North,Missed Collection (All Materials),2 Bulk-Missed Collection,Sidewalk,11236.0,244 EAST 89 STREET,EAST 89 STREET,AVENUE B,DITMAS AVENUE,NaN,NaN,ADDRESS,BROOKLYN,DSNY Garage,The Department of Sanitation picked up the items and determined that the missed collection complaint was not warranted because items were placed out at the wrong location.,17 BROOKLYN,3.07913e+09,BROOKLYN,PHONE,Unspecified,BROOKLYN,40.6482,-73.9161,2010-07-09 12:00:00,1.0,Summer
1430,17608991,2010-07-08 08:16:00,DSNY,BCC - Brooklyn North,Sanitation Condition,12 Dead Animals,Sidewalk,11210.0,1635 ALBANY AVENUE,ALBANY AVENUE,GLENWOOD ROAD,AVENUE H,NaN,NaN,ADDRESS,BROOKLYN,DSNY Garage,The Department of Sanitation cleaned the location.,17 BROOKLYN,3.07725e+09,BROOKLYN,PHONE,Unspecified,BROOKLYN,40.6333,-73.9371,2010-07-08 12:00:00,0.0,Summer


In [49]:
clean_311[clean_311['facility_type'] == "School"].head()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,facility_type,resolution_description,community_board,bbl,borough,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,resolved_date,days_to_close,season
4412,17613598,2010-07-09 15:57:05,DOE,School - MS 322,School Maintenance,Air Conditioning Problem,School,10040.0,4600 BROADWAY,BROADWAY,ELLWOOD STREET,WEST 196 STREET,NaN,NaN,NaN,NEW YORK,School,NaN,12 MANHATTAN,1.02172e+09,MANHATTAN,PHONE,School - MS 322,MANHATTAN,40.8608,-73.9307,2010-07-15 12:53:48,6.0,Summer
4513,17613777,2010-07-09 15:50:28,DOE,School - PS-IS 295,School Maintenance,Air Conditioning Problem,School,11428.0,222-14 JAMAICA AVENUE,JAMAICA AVENUE,224 STREET,222 STREET,NaN,NaN,NaN,QUEENS VILLAGE,School,NaN,13 QUEENS,4.10813e+09,QUEENS,PHONE,School - PS-IS 295,QUEENS,40.7206,-73.732,2010-07-14 13:58:34,5.0,Summer
4608,17613940,2010-07-09 23:28:00,DOE,School - PS 20 Clinton Hill,School Maintenance,Other School Condition,School,11205.0,225 ADELPHI STREET,ADELPHI STREET,DE KALB AVENUE,WILLOUGHBY AVENUE,NaN,NaN,NaN,BROOKLYN,School,NaN,02 BROOKLYN,3.02091e+09,BROOKLYN,PHONE,School - PS 20 Clinton Hill,BROOKLYN,40.6908,-73.9714,2010-07-28 14:26:49,19.0,Summer
4652,17614015,2010-07-09 15:15:42,DOE,School - PS-IS 295,School Maintenance,Air Conditioning Problem,School,11428.0,222-14 JAMAICA AVENUE,JAMAICA AVENUE,224 STREET,222 STREET,NaN,NaN,NaN,QUEENS VILLAGE,School,NaN,13 QUEENS,4.10813e+09,QUEENS,PHONE,School - PS-IS 295,QUEENS,40.7206,-73.732,2010-07-14 13:58:34,5.0,Summer
4873,17614441,2010-07-09 17:14:56,DOE,School - PS-IS 295,School Maintenance,Air Conditioning Problem,School,11428.0,222-14 JAMAICA AVENUE,JAMAICA AVENUE,224 STREET,222 STREET,NaN,NaN,NaN,QUEENS VILLAGE,School,NaN,13 QUEENS,4.10813e+09,QUEENS,PHONE,School - PS-IS 295,QUEENS,40.7206,-73.732,2010-07-14 13:58:34,5.0,Summer


Again, there's nothing in the `facility_type` column to suggest that we need to clean or correct the data.  What about `open_data_channel_type`?

In [50]:
clean_311['open_data_channel_type'].value_counts()

UNKNOWN    1596180
PHONE       958114
ONLINE      133622
OTHER        86792
Name: open_data_channel_type, dtype: int64

OK, that's helpful!  And kind of a neat category to work with.  What kinds of complaints do people phone in, and what kinds of complaints do people do via an app or website?  At any rate, we should leave this alone, it's valuable data.  Let's take a look at some of the borough data that has more categories than we expect:

In [51]:
clean_311['borough'].value_counts()

BROOKLYN         732957
QUEENS           570822
MANHATTAN        467182
Unspecified      464594
BRONX            411126
STATEN ISLAND    128027
Name: borough, dtype: int64

Aha!  Unspecified.  That makes perfect sense!  I bet the same is true for `park_borough` as well:

In [52]:
clean_311['park_borough'].value_counts()

BROOKLYN         732957
QUEENS           570822
MANHATTAN        467182
Unspecified      464594
BRONX            411126
STATEN ISLAND    128027
Name: park_borough, dtype: int64

## Conclusion

We have a cleaner, smaller, more precise set of data to work with now, in `clean_311`.  Let's take a peek at its dimensions, values, and scope.

In [53]:
clean_311.shape

(2774723, 29)

In [54]:
data311.shape

(3000000, 50)

We've reduced our data by 21 columns and around 300k rows.  Let's look at the numerical description of our data as well as the 'info' about our data:

In [55]:
clean_311.describe()

,days_to_close
count,2.774672e+06
mean,1.916779e+01
std,9.445253e+01
min,0.000000e+00
25%,1.000000e+00
50%,4.000000e+00
75%,1.200000e+01
max,3.260000e+03


In [56]:
clean_311.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2774723 entries, 0 to 2999999
Data columns (total 29 columns):
unique_key                2774708 non-null object
created_date              2774708 non-null datetime64[ns]
agency                    2774708 non-null object
agency_name               2774708 non-null object
complaint_type            2774708 non-null object
descriptor                2767343 non-null object
location_type             2151083 non-null object
incident_zip              2774708 non-null object
incident_address          2340886 non-null object
street_name               2340879 non-null object
cross_street_1            2442519 non-null object
cross_street_2            2441708 non-null object
intersection_street_1     425507 non-null object
intersection_street_2     425533 non-null object
address_type              2760584 non-null object
city                      2774396 non-null object
facility_type             576516 non-null object
resolution_description    264995

We've done a lot of data cleaning, and this is a great start for our next stage.  We'll save our data locally (although we won't write it to GitHub because it's so big!):

In [57]:
clean_311.to_csv("clean_311_data.csv", index = False)